In [1]:
import pandas as pd
import pickle
import numpy as np
import glob
import os
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold,train_test_split
from sklearn.model_selection import TimeSeriesSplit,KFold, StratifiedKFold
import lightgbm as lgb
import xgboost as xgb
from time import time

/Users/macbookair/anaconda3/envs/omdena/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
solar = pd.concat(map(pd.read_csv,glob.glob(os.path.join("","data/Nigerian States/*.csv"))))   
solar =solar[['LAT','LON','YEAR','MO','DY','RH2M','PS','T2M']]
solar = solar[(solar['RH2M']>=0)&(solar['PS']>=0)&(solar['T2M']>=0)]

/Users/macbookair/anaconda3/envs/omdena/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [3]:
year = solar[['LAT','LON','YEAR']].drop_duplicates()
year['rank'] = year.groupby(['LAT','LON'])[['YEAR']].rank(method='dense',ascending=False)
year = year[year['rank']<=5]

In [ ]:
solar = year.merge(solar, how='left')
solar = solar.reset_index()

In [ ]:
os.chdir('/Users/macbookair/Downloads/GitHub/renewable/Simple Solar Energy Predictor')

In [ ]:
# Load the model
models = []
for i in range(5):
    clf = pickle.load(open(f"XGBoost_KFold_GPU_model_{i}.dat", 'rb'))
    models.append(clf)

In [ ]:
os.chdir('/Users/macbookair/Downloads/GitHub/renewable/task5_estimate_energy_availability/solar potential')

In [ ]:
pred_values = []
for lat, lon, year, mo, dy,rh,ps,tm in zip(solar['LAT'], solar['LON'], solar['YEAR'],
                                   solar['MO'], solar['DY'], solar['RH2M'],
                                   solar['PS'], solar['T2M']):
    test_df = pd.DataFrame([[lat, lon, year, mo, dy,rh,ps,tm]])
    test_df.columns = ['LAT', 'LON', 'YEAR', 'MO', 'DY', 'RH2M', 'PS', 'T2M']
    pred = sum([clf.predict(test_df) for clf in models])/5
    pred_values.append(pred)

In [ ]:
pred_values = pd.DataFrame(pred_values)
pred_values.columns = ['pred_solar']
solar_irr = solar.merge(pred_values,left_index=True, right_index=True, how = 'inner')

In [ ]:
solar_irr.to_csv('pred_5.csv',index=False)